In [30]:
# ======================================
# 📦 Imports & Environment Setup
# ======================================
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from dotenv import load_dotenv
import os
import requests

load_dotenv()

# ------------------------------
# 🔑 Load SERPER API Key
# ------------------------------
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
if not SERPER_API_KEY:
    raise ValueError("❌ SERPER_API_KEY missing in .env file!")


# ------------------------------
# 🤖 LLM (GPT-4o Mini)
# ------------------------------
llm = LLM(
    model="openai/gpt-4o-mini",
    temperature=0.1
)


# ======================================
# 🔍 Serper Search Tool
# ======================================
@tool("web_search")
def serper_search(query: str) -> str:
    """Search the web using Serper.dev."""
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}

    payload = {"q": query, "num": 5, "gl": "us", "hl": "en"}

    response = requests.post(url, json=payload, headers=headers)
    return str(response.json())
# ======================================
# Researcher 1 — Renewable Energy
# ======================================
renewable_agent = Agent(
    name="RenewableEnergyResearcher",
    role="Energy Research Specialist",
    backstory=(
        "You are an experienced energy analyst who specializes in evaluating "
        "renewable technologies. You rely strictly on verified search results "
        "and avoid adding external information."
    ),
    goal=(
        "Research the latest advancements in renewable energy using Serper search "
        "and produce a factual 1–2 sentence summary."
    ),
    tools=[serper_search],
    llm=llm
)

# ======================================
# Researcher 2 — Electric Vehicles
# ======================================
ev_agent = Agent(
    name="EVResearcher",
    role="Electric Vehicle Analyst",
    backstory=(
        "You analyze trends in electric vehicle technology, maintaining strict "
        "grounding in verifiable search results."
    ),
    goal=(
        "Research the latest EV advancements using Serper and produce a factual "
        "1–2 sentence summary."
    ),
    tools=[serper_search],
    llm=llm
)

# ======================================
# Researcher 3 — Carbon Capture
# ======================================
carbon_agent = Agent(
    name="CarbonCaptureResearcher",
    role="Carbon Capture Specialist",
    backstory=(
        "You evaluate emerging methods in carbon capture and climate engineering."
    ),
    goal=(
        "Research carbon capture advancements using Serper and produce a factual "
        "1–2 sentence summary."
    ),
    tools=[serper_search],
    llm=llm
)
renewable_task = Task(
    description="Research recent advancements in *renewable energy sources*.",
    expected_output="A factual 1–2 sentence summary.",
    agent=renewable_agent,
)

ev_task = Task(
    description="Research recent advancements in *electric vehicle technology*.",
    expected_output="A factual 1–2 sentence summary.",
    agent=ev_agent,
)

carbon_task = Task(
    description="Research current *carbon capture* advancements.",
    expected_output="A factual 1–2 sentence summary.",
    agent=carbon_agent,
)
synthesis_agent = Agent(
    name="SynthesisAgent",
    role="Technical Synthesis Writer",
    backstory=(
        "You combine multiple research summaries into a structured analysis, "
        "ensuring coherence without introducing new information."
    ),
    goal=(
        "Integrate the three research summaries into a clean, structured, factual "
        "report without adding any new content."
    ),
    llm=llm
)
synthesis_task = Task(
    description=(
        "Synthesize the following research results into a structured analysis:\n\n"
        f"- Renewable Energy: {{renewable_task}}\n"
        f"- Electric Vehicles: {{ev_task}}\n"
        f"- Carbon Capture: {{carbon_task}}\n\n"
        "Follow the exact format:\n"
        "## Summary of Recent Sustainable Technology Advancements\n\n"
        "### Renewable Energy Findings\n"
        "{renewable_task}\n\n"
        "### Electric Vehicle Findings\n"
        "{ev_task}\n\n"
        "### Carbon Capture Findings\n"
        "{carbon_task}\n\n"
        "### Overall Conclusion\n"
        "A 1–2 sentence conclusion based ONLY on the above findings."
    ),
    expected_output="A structured report following the exact template.",
    agent=synthesis_agent,
    context=[renewable_task, ev_task, carbon_task]
)
crew = Crew(
    agents=[renewable_agent, ev_agent, carbon_agent, synthesis_agent],
    tasks=[
         renewable_task, ev_task, carbon_task,   
        synthesis_task                            
    ],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print("\n\n=== FINAL SYNTHESIS REPORT ===\n")
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5e154506-7ebc-4ca5-bdca-7f05813be902                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Energy Research Specialist                                                                              │
│                                                                                                                 │
│  Task: Research recent advancements in *renewable energy sources*.                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Energy Research Specialist                                                                              │
│                                                                                                                 │
│  Thought: I need to gather information on recent advancements in renewable energy sources.                      │
│                                                                                                                 │
│  Using Tool: web_search                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "query": "recent advancements in renewable energy sources"                                                   │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Synthesis Writer                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Summary of Recent Sustainable Technology Advancements                                                       │
│                                                                                                                 │
│  ### Renewable Energy Findings                                                                                  │
│  Recent advancements in renewable energy include the development of advanced energy storage solutions, green    │
│  hydrogen production, and the integration of AI and big data in energy systems. Additionally, innovations such  │
│  as floating solar farms and offshore wind expansion are gaining traction, making renewable energy sources      │
│  more efficient and accessible.                                                                                 │
│                                                                                                                 │
│  ### Electric Vehicle Findings                                                                                  │
│  Recent advancements in electric vehicle technology include the rise of bidirectional charging and              │
│  vehicle-to-grid (V2G) integration, allowing EVs to contribute to grid stability by providing bi-directional    │
│  charging. Additionally, trends such as AI-powered health checks, advanced battery technologies, and improved   │
│  connectivity are shaping the future of electric mobility.                                                      │
│                                                                                                                 │
│  ### Carbon Capture Findings                                                                                    │
│  In 2023, the announced capture capacity for carbon dioxide is projected to increase by 35%, while the          │
│  announced storage capacity is expected to rise by 70%, bringing the total potential CO2 capture to             │
│  approximately 435 million tonnes per year and storage capacity to around 615 million tonnes per year. These    │
│  advancements reflect significant progress in carbon capture utilization and storage (CCUS) technologies aimed  │
│  at mitigating climate change.                                                                                  │
│                                                                                                                 │
│  ### Overall Conclusion                                                                                         │
│  The advancements in renewable energy, electric vehicles, and carbon capture technologies collectively          │
│  indicate a significant movement towards sustainable solutions that can effectively address climate change and  │
│  enhance energy efficiency.                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6858e72c-e2fb-470d-af71-11fe3b3a8f54                                                                     │
│  Agent: Technical Synthesis Writer                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5e154506-7ebc-4ca5-bdca-7f05813be902                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Summary of Recent Sustainable Technology Advancements                                         │
│                                                                                                                 │
│  ### Renewable Energy Findings                                                                                  │
│  Recent advancements in renewable energy include the development of advanced energy storage solutions, green    │
│  hydrogen production, and the integration of AI and big data in energy systems. Additionally, innovations such  │
│  as floating solar farms and offshore wind expansion are gaining traction, making renewable energy sources      │
│  more efficient and accessible.                                                                                 │
│                                                                                                                 │
│  ### Electric Vehicle Findings                                                                                  │
│  Recent advancements in electric vehicle technology include the rise of bidirectional charging and              │
│  vehicle-to-grid (V2G) integration, allowing EVs to contribute to grid stability by providing bi-directional    │
│  charging. Additionally, trends such as AI-powered health checks, advanced battery technologies, and improved   │
│  connectivity are shaping the future of electric mobility.                                                      │
│                                                                                                                 │
│  ### Carbon Capture Findings                                                                                    │
│  In 2023, the announced capture capacity for carbon dioxide is projected to increase by 35%, while the          │
│  announced storage capacity is expected to rise by 70%, bringing the total potential CO2 capture to             │
│  approximately 435 million tonnes per year and storage capacity to around 615 million tonnes per year. These    │
│  advancements reflect significant progress in carbon capture utilization and storage (CCUS) technologies aimed  │
│  at mitigating climate change.                                                                                  │
│                                                                                                                 │
│  ### Overall Conclusion                                                                                         │
│  The advancements in renewable energy, electric vehicles, and carbon capture technologies collectively          │
│  indicate a significant movement towards sustainable solutions that can effectively address climate change and  │
│  enhance energy efficiency.                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



=== FINAL SYNTHESIS REPORT ===

## Summary of Recent Sustainable Technology Advancements

### Renewable Energy Findings
Recent advancements in renewable energy include the development of advanced energy storage solutions, green hydrogen production, and the integration of AI and big data in energy systems. Additionally, innovations such as floating solar farms and offshore wind expansion are gaining traction, making renewable energy sources more efficient and accessible.

### Electric Vehicle Findings
Recent advancements in electric vehicle technology include the rise of bidirectional charging and vehicle-to-grid (V2G) integration, allowing EVs to contribute to grid stability by providing bi-directional charging. Additionally, trends such as AI-powered health checks, advanced battery technologies, and improved connectivity are shaping the future of electric mobility.

### Carbon Capture Findings
In 2023, the announced capture capacity for carbon dioxide is projected to increase by 35%